# Recherche de production solaire

In [89]:
import pandas as pd
import requests
import numpy as np
import datetime as dt

df = pd.read_csv("data/prod_solaire.csv")

df

,date,prod_solaire
0,2016-09-01,77.163310
1,2016-09-02,72.371632
2,2016-09-03,31.906049
3,2016-09-04,22.494454
4,2016-09-05,68.105196
...,...,...
3307,2025-09-25,50.075232
3308,2025-09-26,44.815599
3309,2025-09-27,46.526905
3310,2025-09-28,NaN


In [ ]:
df.loc[(df.iloc[:, 1] < 100)  & (df.iloc[:, 1] > 0)]
types = ["hydro", "eolienne", "solar"]
print(f"Please enter any of types in this list : {types} ")

Please enter any of types in this list : ['hydro', 'eolienne', 'solar'] 


In [91]:
df.describe()

,prod_solaire
count,3310.000000
mean,47.797643
std,30.228459
min,0.000000
25%,23.341965
50%,43.828634
75%,71.486126
max,551.908083


In [93]:
df_missing = df.isnull().sum()

display(df_missing)

df_ext = df.sort_values(by=["prod_solaire"], ascending=True)


date            0
prod_solaire    2
dtype: int64

In [98]:
df["date"] = pd.to_datetime(df["date"])
df.loc[df["prod_solaire"] > 0].sort_values(by="prod_solaire")

df["date"].dt.dayofweek.value_counts()



date
3    474
4    474
5    474
0    473
2    473
6    472
1    472
Name: count, dtype: int64

In [ ]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 43.6109,
	"longitude": 3.8763,
	"start_date": "2016-09-01",
	"end_date": "2025-09-25",
	"hourly": ["global_tilted_irradiance", "temperature_2m", "relative_humidity_2m", "precipitation", "wind_speed_10m", "wind_direction_10m"],
	"tilt": 35,
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_global_tilted_irradiance = hourly.Variables(0).ValuesAsNumpy()
hourly_temperature_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["global_tilted_irradiance"] = hourly_global_tilted_irradiance
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_global_tilted_irradiance)
gti_df = hourly_dataframe["global_tilted_irradiance"]

print(gti_df.info())
print(gti_df.describe())

Coordinates: 43.620384216308594°N 3.930722951889038°E
Elevation: 58.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
 [0. 0. 0. ... 0. 0. 0.]
<class 'pandas.core.series.Series'>
RangeIndex: 79488 entries, 0 to 79487
Series name: global_tilted_irradiance
Non-Null Count  Dtype  
--------------  -----  
79488 non-null  float32
dtypes: float32(1)
memory usage: 310.6 KB
None
count    79488.000000
mean       207.878891
std        296.646332
min          0.000000
25%          0.000000
50%          8.355879
75%        383.465149
max       1074.020630
Name: global_tilted_irradiance, dtype: float64
